# Data Wrangling o procesos ETL

## Resumen

En esta notebook se realiza una inspección inicial del dataset, con la idea de identificar la estructura, tipos de variables, y demás. Se identifica también, si en el conjunto de datos hay datos faltantes o registros duplicados, y se actúa en consecuencia.

## 1. Inicialización

### 1.1. Importación de librerías

In [2]:
import pandas as pd
import numpy as np
import os
#
# separador multi-OS
separador=os.sep

In [2]:
### 1.2. El Dataset

In [3]:
df=pd.read_csv('datasets'+str(separador)+'iris.csv')

## 2. Descripción del Dataset

### 2.1. Tamaño

In [4]:
df.shape

(150, 5)

### 2.2. Primeros registros

In [5]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


### 2.3. Lista de variables

### 2.3. Tipos de variables

In [6]:
tipos = df.columns.to_series().groupby(df.dtypes).groups
print(tipos)

{float64: ['sepal.length', 'sepal.width', 'petal.length', 'petal.width'], object: ['variety']}


In [7]:
variables=df.columns.tolist()
print('variables=',variables)
ctext = tipos[np.dtype('object')]
print('Variables Cualitativas: ',ctext)
cnum = list(set(variables) - set(ctext))
print('Variables Numéricas: ',cnum)

variables= ['sepal.length', 'sepal.width', 'petal.length', 'petal.width', 'variety']
Variables Cualitativas:  Index(['variety'], dtype='object')
Variables Numéricas:  ['sepal.width', 'petal.length', 'sepal.length', 'petal.width']


### 2.4. Ver si hay datos faltantes

In [8]:
print(df.isna().sum())

sepal.length    0
sepal.width     0
petal.length    0
petal.width     0
variety         0
dtype: int64


### 2.5. Ver si hay registros duplicados

In [9]:
print(df.duplicated().value_counts())

False    149
True       1
dtype: int64


## 3. Limpieza del Dataset

### 3.1. Completar datos faltantes

En caso que en algún registro, aparezca un dato faltante (NAN), la mayoría de los algoritmos no podrá operar con ellos. Es por ello que se debe tomar una decisión: Eliminar el registro o bien reemplazar ese valor por algún valor pre-establecido como ser el promedio.

In [11]:
respuesta=input('En caso de NAN: ¿(R) Reemplazar / (E) Eliminar registro?')
if(respuesta=='R'):
    hayFaltantes=df.isnull().any().any()
    print('Hay datos vacios? ',hayFaltantes)
    # Si hay faltantes: competo a los numéricos con el promedio:
    for c in cnum:
        mean = df[c].mean()
        df[c] = df[c].fillna(mean)
    # Si hay faltantes cuantitativas completo:
    def completarVariableCualitativa(df,variable):
        var=[]
        for i in range(len(df)):
            if(pd.isna(df.iloc[i,variable])):
                dato='no_data'
            else:
                dato=df.iloc[i,variable]
            var.append(dato)
        return var
    ctext = tipos[np.dtype('object')]
    ctextnum=[]
    for i in range(len(ctext)):
        for j in range(len(variables)):
            if ctext[i]==variables[j]:
                ctextnum.append(j)
    for i in range(len(ctext)):
        completarVariableCualitativa(df,int(ctextnum[i]))
elif(respuesta=='E'):
    df = df.dropna(axis = 0, how ='any')
else:
    print('ERROR DESCONOCIDO')

En caso de NAN: ¿(R) Reemplazar / (E) Eliminar registro?E


### 3.2. Eliminar registros duplicados

Si hubiera registros duplicados, puede que sea un error o bien que haya dos elementos de la muestra exactamente iguales. Pueden mantenerse o eliminarse. En este caso, ante duplicados, se decide eliminarlos.

In [12]:
df_sin_duplicados = df.drop_duplicates(keep='first')
df_sin_duplicados.shape

(149, 5)

Observar la cantidad de registros antes de este proceso y después: antes había 150 registros, de los cuales se había identificado que uno era duplicado. Tras su eliminación, observar que shape muestra 149 registros. <br>
<b>Importante: </b> drop_duplicates() por defecto elimina todos los registros que aparecen duplicados. Normalmente es deseable retener uno de ellos. Por eso: "keep='first'".

### 3.3. Normalizar Dataset

Muchos algoritmos requieren que los números se encuentren en el mismo orden de magnitud, de modo de no asignar pesos fuertes a variables que símplemente tienen valores numéricos altos en comparación con otras. Para ello, normalmente es conveniente la normalización de todo el conjunto de datos (salvo la variable objetivo). Normalmente, tras la normalización, todos los dato pasan a estar entre -1 y 1 o parecido.

In [13]:
def normalizar(df,var):
    import sklearn
    from sklearn import preprocessing
    campos=df.columns.tolist()
    target=campos[var]
    X = np.array(df.drop([target],1))
    y = np.array(df[target])
    X_Norm=sklearn.preprocessing.scale(X)
    df2=pd.DataFrame(X_Norm)
    df2['target']=y
    df2.columns=campos
    return df2
df=normalizar(df,4)

/tmp/ipykernel_5757/222317414.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df.drop([target],1))


### 3.4. Discretizar Dataset

La variable "Variety" presenta los valores: "Setosa", "Virginica" y "Versicolor". Como son del tipo texto, no pueden operarse en la mayoría de los algoritmos. El proceso de discretización crea una variable "Variety_Setosa" y en aquellos registros donde la variedad es "Setosa", en la columna "Variety_Setosa" coloca un 1, y resto en otro caso. Así sucesivamente.

In [14]:
df_esp = pd.get_dummies(df, columns=['variety'])
# Conteo de resultados
print(df_esp.value_counts())

sepal.length  sepal.width  petal.length  petal.width  variety_Setosa  variety_Versicolor  variety_Virginica
-0.052506     -0.822570     0.762758      0.922303    0               0                   1                    2
 0.432165     -1.973554     0.421734      0.395774    0               1                   0                    1
              -0.362176     0.308059      0.132510    0               1                   0                    1
               0.788808     0.933271      1.448832    0               0                   1                    1
 0.553333     -1.743357     0.364896      0.132510    0               1                   0                    1
                                                                                                              ..
-0.537178      1.939791    -1.397064     -1.052180    1               0                   0                    1
                           -1.169714     -1.052180    1               0                   0          